# Notebook: Train Model

* To-Do: Error throwen, wenn es bei den labels ein invalide sequence übersetzung gibt

## Packages

In [1]:
from load_dataset_folds import load_dataset_folds
from transformers import set_seed
from TASD import train_TASD_model
from ACD import train_ACD_model
from OTE import train_OTE_model
import pandas as pd
import numpy as np
import constants
import warnings
import shutil
import torch
import json
import sys

## Parameters

In [2]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "aspect_category" # "aspect_term", "aspect_category", "target_aspect_sentiment_detection"
LLM_SAMPLING = "random"

## Settings

In [3]:
# Set seeds
torch.device(constants.DEVICE)
torch.manual_seed(constants.RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(constants.RANDOM_SEED)
set_seed(constants.RANDOM_SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(constants.RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning,
                        module="transformers.optimization")

# Disable Pycache
sys.dont_write_bytecode = True

## Code

In [4]:
train_dataset, test_dataset = load_dataset_folds(LLM_NAME, N_REAL, N_SYNTH, LLM_SAMPLING)

500 5
500 5


### Load Model

In [5]:
if TARGET == "aspect_category":
   results = train_ACD_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/160 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5477, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.19867871701717377, 'eval_hamming_loss': 0.0, 'eval_accuracy': 1.0, 'eval_f1_macro': 1.0, 'eval_f1_micro': 1.0, 'eval_f1_weighted': 1.0, 'eval_precision_GENERAL-IMPRESSION': 1.0, 'eval_recall_GENERAL-IMPRESSION': 1.0, 'eval_f1_GENERAL-IMPRESSION': 1.0, 'eval_precision_FOOD': 1.0, 'eval_recall_FOOD': 1.0, 'eval_f1_FOOD': 1.0, 'eval_precision_SERVICE': 1.0, 'eval_recall_SERVICE': 1.0, 'eval_f1_SERVICE': 1.0, 'eval_precision_AMBIENCE': 1.0, 'eval_recall_AMBIENCE': 1.0, 'eval_f1_AMBIENCE': 1.0, 'eval_precision_PRICE': 1.0, 'eval_recall_PRICE': 1.0, 'eval_f1_PRICE': 1.0, 'eval_runtime': 2.5237, 'eval_samples_per_second': 198.125, 'eval_steps_per_second': 12.68, 'epoch': 5.0}
{'train_runtime': 78.392, 'train_samples_per_second': 31.891, 'train_steps_per_second': 2.041, 'train_loss': 0.34914917647838595, 'epoch': 5.0}


  0%|          | 0/32 [00:00<?, ?it/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/160 [00:00<?, ?it/s]

In [ ]:
if TARGET == "aspect_term":
    results = train_OTE_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [ ]:
if TARGET == "target_aspect_sentiment_detection":
    results = train_TASD_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)


### Save Results

In [ ]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [ ]:
results

{'LLM_NAME': 'Llama13B',
 'N_REAL': 500,
 'N_SYNTH': 0,
 'TARGET': 'aspect_category',
 'LLM_SAMPLING': 'random',
 'eval_loss': 0.5149109959602356,
 'eval_accuracy': 0.134,
 'eval_hamming_loss': 0.1988,
 'eval_f1_macro': 0.1604790419161677,
 'eval_f1_micro': 0.3503267973856209,
 'eval_f1_weighted': 0.25432494756920393,
 'eval_precision_GENERAL-IMPRESSION': 0.0,
 'eval_recall_GENERAL-IMPRESSION': 0.0,
 'eval_f1_GENERAL-IMPRESSION': 0.0,
 'eval_precision_FOOD': 0.0,
 'eval_recall_FOOD': 0.0,
 'eval_f1_FOOD': 0.0,
 'eval_precision_SERVICE': 1.0,
 'eval_recall_SERVICE': 0.67,
 'eval_f1_SERVICE': 0.8023952095808384,
 'eval_precision_AMBIENCE': 0.0,
 'eval_recall_AMBIENCE': 0.0,
 'eval_f1_AMBIENCE': 0.0,
 'eval_precision_PRICE': 0.0,
 'eval_recall_PRICE': 0.0,
 'eval_f1_PRICE': 0.0,
 'runtime': 19.87282395362854,
 'runtime_formatted': '19s'}

### Remove useless folders

In [ ]:
try:
    shutil.rmtree("outputs")
except:
    pass